# Workflow for simulations of an architectured crop

In [6]:
#| echo: false

# Ignore warning about depreciated modules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [4]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from openalea.plantgl.all import Vector3
from oawidgets.plantgl import *

## 1. Parametrized architectural model

ArchiCrop is a Structural Plant Parametric Model (SPPM) which represents the 3D+t visible architecture of a plant shoot. 

An SPPM is a 3D architectural plant model that can be described at different scales : plant, axis, phytomer, organ. 
The topology of a plant is defined by a set of parameters : number of axes, number of phytomers of each axis, etc. 
The geometry of the different organs is defined by a set of parameters : axes and leaves length, insertion angle and curvature, leaves width-to-length ratio, etc. 
Organs development is defined by a set of parameters : phyllochron, "ligulochron" (i.e. time from start to end of growth of an organ), delay of ramification, etc. 

## 2. Allometric laws on architectural parameters 

## 3. Organs growth

### 3.1. Compute start and end of organ growth based of phyllochron and elongation rate

In [5]:
from openalea.archicrop.dynamic import thermal_time

### 3.2. Retrieve growth increments for given thermal time steps

Given a simulation with a crop model (or any source of constraint at the crop scale) :

In [2]:
def read_columns_from_file(filename, separator=';'):
    '''Read a file column-wise and store the values in a 2D-list.'''
    with open(filename, 'r') as file:
        columns = []
        for line in file:
            values = line.strip().split(separator) # Strip any extra whitespace and split the line by the separator
            if not columns:
                columns = [[] for _ in values]
            while len(columns) < len(values):
                columns.append([])
            for i, value in enumerate(values):
                columns[i].append(value)
            # Handle lines with fewer values than columns
            for i in range(len(values), len(columns)):
                columns[i].append('')
    return columns

In [3]:
## e.g. from a STICS simulation for maize

filename = 'mod_smaize.sti'
columns = read_columns_from_file(filename)
columns = columns[5:]

# Cut the simulation at the end of vegetative growth
end = 115 
# To get an equal LA/plant 
density = 0.00095 # density = 9.5 plant/m2 = 0.00095 plant/cm2
la = np.array([float(i)/density for i in columns[0][18:end]]) # leaf area
height = np.array([float(i) for i in columns[1][18:end]]) # height
tt = np.cumsum([float(i) for i in columns[2][18:end]]) # thermal times

### Compute whole plant architectures for all time steps